In [23]:
###################################################################################
# Check that these are the correct paths running (change if necessary)

# Finn's Laptop
dicombasefolder = '/mnt/d/Finn/KPUM_NODDI/DICOM_KPUM_NODDI'

# KPUM Workstation
#dicombasefolder = '/mnt/e/Finn/KPUM_NODDI/DICOM_KPUM_NODDI'

# sourcedata base folder to go within dicom base folder
sourcedatabasefolder = 'sourcedata'

# tsv-files to keep track of in dicombasefolder
subjecttrackerpath = dicombasefolder
subjecttrackerfile = 'Subject_Tracker_for_dicomdir2sourcedata.tsv'

###################################################################################
# User input

# Participant details
subject = '002'     # Subject's study ID
session = 'MR1'     # Current scan's session ID 

# The DICOM folder to be processed
#dicomdatafolder = '002_MR1_8193760_20210512/DICOM_fromPACS'
dicomdatafolder = '002_MR1_8193760_20210512/DICOM_NODDI'
       
###################################################################################

In [24]:
import os, subprocess
import pandas as pd

# Define path and create output dicomdir if not present
outputdicomdir = os.path.join(dicombasefolder, sourcedatabasefolder, f'sub-{subject}', f'ses-{session}')
if not os.path.exists(outputdicomdir): # then make this directory
    os.makedirs(outputdicomdir)

# Read the Subject_Tracker_dicomdir2sourcedata.tsv
subjecttrackertsv = os.path.join(subjecttrackerpath, subjecttrackerfile)
df = pd.read_csv(subjecttrackertsv, sep="\t")
df.fillna('', inplace=True)

# Check if not converted
if  [ df.loc[df['dicomfolder'] == dicomdatafolder, ['converted']] != 'Yes' ] :
    # Run bash-script
    print("Start re-organising and anonymizing DCMs")
    result = subprocess.run(f"bash ../shell/nifti_dicomdir2sourcedata.sh {dicombasefolder} {dicomdatafolder} {subject} {session} -sourcedata {sourcedatabasefolder}", stdout=subprocess.PIPE, shell=True)
    #print(result.stdout)
    # Update Subject_Tracker_dicomdir2sourcedata.tsv
    df.loc[df['dicomfolder'] == dicomdatafolder, ['converted']] = 'Yes'
    df.loc[df['dicomfolder'] == dicomdatafolder, ['sourcedatafolder']] = os.path.join(sourcedatabasefolder, f'sub-{subject}', f'ses-{session}')
    df.to_csv(os.path.join(dicombasefolder, subjecttrackertsv), sep="\t", index=False)
    print("Done re-organising and anonymizing DCMs")
else :
    print("Done re-organising and anonymizing DCMs")


Start re-organising and anonymizing DCMs
